-----------------------------
#### Get GPT embeddings 
- for food reviews data

https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
---------------------------

**Load the Dataset:**

- The dataset used in this example is fine-food reviews from Amazon.
- It contains a total of 568,454 food reviews that Amazon users left up to October 2012.
- We will use a subset of this dataset, consisting of the 1,000 most recent reviews for illustration purposes.
- The reviews are in English and tend to be positive or negative.
- Each review has a ProductId, UserId, Score, review title (Summary), and review body (Text).

**Combine Review Summary and Review Text:**

- We will combine the review summary and review text into a single combined text.
- The model will encode this combined text, and it will output a single vector embedding.

In [3]:
import pandas as pd
import numpy as np

In [5]:
location = r'D:\AI-DATASETS\02-MISC-large\GenAI-LLMs\amazon_food_reviews.csv'

In [7]:
df = pd.read_csv(location)

In [8]:
df.shape

(568454, 10)

In [9]:
df.dtypes

Id                         int64
ProductId                 object
UserId                    object
ProfileName               object
HelpfulnessNumerator       int64
HelpfulnessDenominator     int64
Score                      int64
Time                       int64
Summary                   object
Text                      object
dtype: object

- The dataset contains a total of 568,454 food reviews Amazon users left up to October 2012. 
- We will use a subset of 2,000 most recent reviews for illustration purposes. 
- The reviews are in English and tend to be positive or negative. 
- Each review has a ProductId, UserId, Score, review title (Summary) and review body (Text).

In [11]:
df.sample(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
41946,41947,B006Q7YFLQ,A1BT6TL7UCIC5Y,Quoad Toad,0,0,5,1344038400,My dog loves them,My dog loves these treats. She is not very di...
555067,555068,B0000DJ7SR,AWZ1C2FT0L3W9,Sandy,5,6,5,1156982400,Potatoes,excellent - will order again!! It's nice to b...
208670,208671,B0007Q426U,A1Q1QZ9U5E1QCI,Margaret E. Harrington,0,0,5,1247443200,Lindberg Snider Porterhouse & Roast Seasoning ...,I am NEVER without this product. It ALWAYS ta...


In [12]:
df.Score.describe()

count    568454.000000
mean          4.183199
std           1.310436
min           1.000000
25%           4.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: Score, dtype: float64

In [1]:
df.Score.hist()

NameError: name 'df' is not defined

We will combine the review summary and review text into a single combined text. The model will encode this combined text and it will output a single vector embedding.

In [ ]:
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()

df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)

In [ ]:
df.shape

In [ ]:
# subsample to 2k most recent reviews and remove samples that are too long
top_n = 1000

df    = df.sort_values("Time").tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out
df.drop("Time", axis=1, inplace=True)

In [ ]:
df.shape

In [ ]:
df.sample(3)

In [ ]:
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)

In [ ]:
review = df.combined.sample(3).values

In [ ]:
review

#### About tiktoken

- `tiktoken` is a fast open-source tokenizer by OpenAI. It is used to tokenize text into a sequence of integers that can be processed by OpenAI's GPT models. This is necessary because GPT models do not understand natural language, they only understand sequences of integers.

- Here are some of the benefits of using tiktoken:

    - It is `fast and efficient`. tiktoken is a highly optimized tokenizer that can tokenize text at a high rate. This is important for applications that require real-time processing of text.
    - It is `accurate`. tiktoken produces accurate tokenizations that capture the meaning of the original text. This is important for applications that require high-quality results, such as machine translation and natural language generation.
    - It is `easy to use`. tiktoken has a simple and easy-to-use API that makes it easy to integrate into existing applications.

In [ ]:
#!pip install tiktoken

In [ ]:
import tiktoken

**Tokenize a text string:**

- tiktoken.get_encoding 
    - function is used to load an encoding by name. 
    - The first time this function is called for a given encoding name, it will download the encoding from the internet. 
    - Later calls will use the cached version of the encoding.
    - The `tiktoken.get_encoding` function takes one argument, which is the encoding name. 
    - The encoding name is a string that identifies the encoding. 
        - For example, the encoding name "cl100k_base" identifies the BPE encoding that was trained on the cl100k dataset.
        
- The `tiktoken.get_encoding` function returns an Encoding object. 
    - The Encoding object has the following methods:

        - `encode`(text): Encodes a text string into a sequence of integers.
        - `decode`(tokens): Decodes a sequence of integers into a text string.
        - `vocab_size()`: Returns the size of the vocabulary.
        - `eos()`: Returns the ID of the end-of-sentence token.
        - `pad()`: Returns the ID of the pad token.

In [ ]:
# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4")
enc

In [ ]:
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
enc

In [ ]:
# # this the encoding for text-embedding-ada-002
encoding = tiktoken.get_encoding("cl100k_base")

tokens   = encoding.encode("This is a text string at doddaballapura road.")
print(tokens)

**Decode a sequence of tokens**

In [ ]:
text = encoding.decode(tokens)
print(text)

In [ ]:
num_tokens = len(tokens)
print(num_tokens)

... back to code

In [49]:
import os
import openai
from openai import OpenAI

In [51]:
df

,ProductId,UserId,Score,Summary,Text,combined
467784,B000V1O28Y,A1CHFK7Z8ZJBQ7,4,Better than I thought they would be!,These were actually quite tasty despite the ex...,Title: Better than I thought they would be!; C...
102286,B001QEB3E6,AY2O7I9BNG2UI,4,"Dog loves them, but I hate the smell","I have a 70 pound lab mix, and he goes nuts fo...","Title: Dog loves them, but I hate the smell; C..."
228586,B00295IGHS,A37VSXI1MDHBWJ,2,Too runny for my taste,"I bought this to use to give my dogs pills, so...",Title: Too runny for my taste; Content: I boug...
149371,B006BXUKAA,A2Y2RTN8AZ9C7F,2,Smells like the beans are flavored with alcohol,"When I opened the back, the overwhelming alcoh...",Title: Smells like the beans are flavored with...
175998,B007TGO1U8,A2FHXIKEYLW2C0,1,Aftertaste,"I am disappointed in this product. First, ther...",Title: Aftertaste; Content: I am disappointed ...
...,...,...,...,...,...,...
7177,B004OQLIHK,AKHQMSUORSA91,5,Delicious!,I have ordered these raisins multiple times. ...,Title: Delicious!; Content: I have ordered the...
401971,B0006349W6,A21BT40VZCCYT4,5,Good Training Treat,My dog will come in from outside when I am tra...,Title: Good Training Treat; Content: My dog wi...
462087,B00611F084,A6D4ND3C3BCYV,5,Jamica Me Crazy Coffee,Wolfgang Puck's Jamaica Me Crazy is that wonde...,Title: Jamica Me Crazy Coffee; Content: Wolfga...
267548,B005QKH5HA,A3LR9HCV3D96I3,5,Party Peanuts,Great product for the price. Mix with the Asia...,Title: Party Peanuts; Content: Great product f...


In [53]:
# embedding model parameters
embedding_model    = "text-embedding-ada-002" # smallest
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002

max_tokens         = 8000  # the maximum for text-embedding-ada-002 is 8191

In [55]:
encoding = tiktoken.get_encoding(embedding_encoding)

In [57]:
# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))

In [36]:
df

,ProductId,UserId,Score,Summary,Text,combined,n_tokens
467784,B000V1O28Y,A1CHFK7Z8ZJBQ7,4,Better than I thought they would be!,These were actually quite tasty despite the ex...,Title: Better than I thought they would be!; C...,58
102286,B001QEB3E6,AY2O7I9BNG2UI,4,"Dog loves them, but I hate the smell","I have a 70 pound lab mix, and he goes nuts fo...","Title: Dog loves them, but I hate the smell; C...",98
228586,B00295IGHS,A37VSXI1MDHBWJ,2,Too runny for my taste,"I bought this to use to give my dogs pills, so...",Title: Too runny for my taste; Content: I boug...,100
149371,B006BXUKAA,A2Y2RTN8AZ9C7F,2,Smells like the beans are flavored with alcohol,"When I opened the back, the overwhelming alcoh...",Title: Smells like the beans are flavored with...,88
175998,B007TGO1U8,A2FHXIKEYLW2C0,1,Aftertaste,"I am disappointed in this product. First, ther...",Title: Aftertaste; Content: I am disappointed ...,79
...,...,...,...,...,...,...,...
7177,B004OQLIHK,AKHQMSUORSA91,5,Delicious!,I have ordered these raisins multiple times. ...,Title: Delicious!; Content: I have ordered the...,43
401971,B0006349W6,A21BT40VZCCYT4,5,Good Training Treat,My dog will come in from outside when I am tra...,Title: Good Training Treat; Content: My dog wi...,48
462087,B00611F084,A6D4ND3C3BCYV,5,Jamica Me Crazy Coffee,Wolfgang Puck's Jamaica Me Crazy is that wonde...,Title: Jamica Me Crazy Coffee; Content: Wolfga...,40
267548,B005QKH5HA,A3LR9HCV3D96I3,5,Party Peanuts,Great product for the price. Mix with the Asia...,Title: Party Peanuts; Content: Great product f...,45


In [1]:
df.n_tokens.hist()

NameError: name 'df' is not defined

In [41]:
# number of samples with number of tokens more than max_tokens
df[df.n_tokens <= 50]

,ProductId,UserId,Score,Summary,Text,combined,n_tokens
413611,B0026KPDG8,A3EJRBPUFFECQP,5,great portion contolled snack,These are great to have around as a snack... i...,Title: great portion contolled snack; Content:...,39
83822,B001EQ5GXK,AS9N6RG8ZDO0U,5,NorCal Grandma,This decaf pod has wonderful flavor. It is mu...,Title: NorCal Grandma; Content: This decaf pod...,40
152786,B000FDMLVI,A1C4UOLGSFKELF,5,Great bread!!,Just made Hodgson Mill white bread this mornin...,Title: Great bread!!; Content: Just made Hodgs...,37
393873,B0059JXK2M,A14ZTUL1511SLP,5,Very Good,"Can't believe these are sugar-free, they reall...",Title: Very Good; Content: Can't believe these...,40
529735,B0032CB15Q,A34YL36NCRLSQT,5,Great Morning Coffee,This is a great morning starter!! If you love...,Title: Great Morning Coffee; Content: This is ...,38
...,...,...,...,...,...,...,...
443040,B001A43E6M,A32UNJZRBQENQW,5,Arizona Green Tea with Pomegranate & Acai,I love the convenience of the mail order proce...,Title: Arizona Green Tea with Pomegranate & Ac...,39
7177,B004OQLIHK,AKHQMSUORSA91,5,Delicious!,I have ordered these raisins multiple times. ...,Title: Delicious!; Content: I have ordered the...,43
401971,B0006349W6,A21BT40VZCCYT4,5,Good Training Treat,My dog will come in from outside when I am tra...,Title: Good Training Treat; Content: My dog wi...,48
462087,B00611F084,A6D4ND3C3BCYV,5,Jamica Me Crazy Coffee,Wolfgang Puck's Jamaica Me Crazy is that wonde...,Title: Jamica Me Crazy Coffee; Content: Wolfga...,40


In [91]:
# number of samples with number of tokens less than max_tokens
df[df.n_tokens <= max_tokens].shape

(2000, 7)

#### Get embeddings and save them for future reuse
- Ensure you have your API key set in your environment

In [3]:
from openai import OpenAI

client = openai.OpenAI(
   
    #api_key=openai_api_key
)

NameError: name 'openai' is not defined

In [43]:
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [40]:
%%time
# This may take a few minutes (10 - 20 mins)
df['ada_embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

CPU times: total: 7.95 s
Wall time: 15min 47s


#### Options for faster obtaining of embeddings

**Option 1** - Batch Processing
- This reduces the overhead associated with making individual API calls and can significantly reduce the total time required.

In [44]:
def get_embeddings_batch(texts, model='text-embedding-ada-002'):
    # Using the client to create embeddings
    response = client.embeddings.create(input=texts, model=model)
    
    # Extract embeddings from the response
    return [embedding.embedding for embedding in response.data]

In [46]:
batch_size = 150  # Adjust based on API limits and memory, 50, 100, 150 ...
embeddings = []

In [47]:
%%time
for i in range(0, len(df), batch_size):
    
    batch = df.combined[i:i+batch_size].tolist()
    
    embeddings.extend(get_embeddings_batch(batch))

df['ada_embedding'] = embeddings

Wall time: 12.9 s


**Option 2** - Parallel Processing with Threading:
- Using multiple threads can help make API calls concurrently, thus utilizing available resources more efficiently.

In [48]:
from concurrent.futures import ThreadPoolExecutor

In [49]:
def get_embeddings_batch(texts, model='text-embedding-ada-002'):
    # Using the client to create embeddings
    response = client.embeddings.create(input=texts, model=model)
    
    # Extract embeddings from the response
    return [embedding.embedding for embedding in response.data]

In [50]:
def process_batch(batch):
    return get_embeddings_batch(batch)

In [51]:
batch_size = 50  # Adjust based on API limits and memory
embeddings = []

In [53]:
# Number of threads to use. Adjust based on your system's capabilities
max_workers = 10

In [54]:
%%time
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    
    # Split the data into batches and process them in parallel
    batches = [df.combined[i:i+batch_size].tolist() for i in range(0, len(df), batch_size)]
    results = executor.map(process_batch, batches)
    
    # Flatten the list of lists into a single list of embeddings
    embeddings = [embedding for batch_result in results for embedding in batch_result]

Wall time: 5.01 s


In [55]:
# Add the embeddings to the DataFrame
df['ada_embedding'] = embeddings

In [56]:
df.sample(5)

,ProductId,UserId,Score,Summary,Text,combined,n_tokens,ada_embedding
535731,B003VXHGDM,A21VWSCGW7UUAR,4,"Good, but not Wolfgang Puck good","Honestly, I have to admit that I expected a li...","Title: Good, but not Wolfgang Puck good; Conte...",178,"[-0.0031321949791163206, -0.010046787559986115..."
538589,B0048KH218,AVDNKJE3E46HW,5,Excellent Value,I discovered quinoa recently. You can read all...,Title: Excellent Value; Content: I discovered ...,57,"[0.0026609161868691444, 0.01671537756919861, 0..."
201428,B001U2HRLM,A1T666SZMKVT5Y,5,Babys Only Toddler Formula Lactose Relief Organic,I have been using this for the last 4 months f...,Title: Babys Only Toddler Formula Lactose Reli...,54,"[-0.013263222761452198, -4.496865221881308e-05..."
405314,B005K4Q4LK,AH1QYCY9W98C5,4,K -Cup Cappuccino,This is a product that we will use again as it...,Title: K -Cup Cappuccino; Content: This is a p...,44,"[-2.4473529265378602e-05, 0.008355854079127312..."
446988,B004HU7TC6,A1BUNS3B1N43T9,5,Better than Western Candy,I love the strawberry flavor of Pocky the best...,Title: Better than Western Candy; Content: I l...,75,"[0.009444493800401688, 0.005671391729265451, 0..."


In [108]:
save_file_path = r'D:\AI-DATASETS\02-MISC-large\GenAI-LLMs\amazon_food_reviews_with_embeddings_2k.csv'
df.to_csv(save_file_path)

To load the data from a saved file, you can run the following:

In [44]:
df_loaded = pd.read_csv(save_file_path)
df_loaded['ada_embedding'] = df_loaded.ada_embedding.apply(eval).apply(np.array)